In [1]:
%load_ext autoreload
%autoreload

import os, sys

modules = os.path.abspath(os.path.join("..", "src"))
sys.path.append(modules)

import polars as pl
from transformers import AutoModel, AutoTokenizer
import nlp_utils, keywords_utils, load_utils, run_analysis
from config import *


/home/msalvetti/notebooks_2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-29 10:18:10.507269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Clean data and merge data

## Firm database
Table with 
|Firm ID|Firm name|url main page|canton|employees|NACE code|cat website|cat patent| n kws web| n kws pat

In [6]:
frame = pl.scan_parquet("../data/cleaned/run_2025-11-21/*.parquet")

In [3]:
# Option 2: Process in batches by ID prefix or chunks
import os

output_dir = "../data/processed/grouped_text/"
os.makedirs(output_dir, exist_ok=True)

# Get unique IDs and process in chunks
unique_ids = frame.select("ID").unique().collect(engine="streaming")["ID"].to_list()
chunk_size = 1000

for i in range(0, len(unique_ids), chunk_size):
    chunk_ids = unique_ids[i:i + chunk_size]
    (
        frame
        .filter(pl.col("ID").is_in(chunk_ids))
        .group_by("ID")
        .agg(pl.col("text").str.concat("\n").alias("text"))
        .with_columns(pl.col("text").str.slice(0, 1_000_000))
        .collect(engine="streaming")
        .write_parquet(f"{output_dir}/chunk_{i // chunk_size:04d}.parquet")
    )
    print(f"Processed chunk {i // chunk_size}")

/tmp/ipykernel_73059/2045756650.py:17: DeprecationWarning: `str.concat` is deprecated; use `str.join` instead. Note also that the default `delimiter` for `str.join` is an empty string, not a hyphen.
  .agg(pl.col("text").str.concat("\n").alias("text"))


Processed chunk 0
Processed chunk 1
Processed chunk 2
Processed chunk 3
Processed chunk 4
Processed chunk 5
Processed chunk 6
Processed chunk 7
Processed chunk 8
Processed chunk 9
Processed chunk 10
Processed chunk 11
Processed chunk 12
Processed chunk 13
Processed chunk 14
Processed chunk 15
Processed chunk 16
Processed chunk 17
Processed chunk 18
Processed chunk 19
Processed chunk 20
Processed chunk 21
Processed chunk 22
Processed chunk 23
Processed chunk 24
Processed chunk 25
Processed chunk 26
Processed chunk 27
Processed chunk 28
Processed chunk 29
Processed chunk 30
Processed chunk 31
Processed chunk 32
Processed chunk 33
Processed chunk 34
Processed chunk 35
Processed chunk 36
Processed chunk 37
Processed chunk 38
Processed chunk 39
Processed chunk 40
Processed chunk 41
Processed chunk 42
Processed chunk 43
Processed chunk 44
Processed chunk 45
Processed chunk 46
Processed chunk 47
Processed chunk 48
Processed chunk 49
Processed chunk 50
Processed chunk 51
Processed chunk 52


In [4]:
data = pl.scan_parquet("../data/cleaned/run_2025-11-21/*.parquet")
kws_matches = pl.read_parquet("../data/processed/kw_web_counts.parquet").explode("cat_web", "n_unique_kw_web").filter(pl.col("n_unique_kw_web")>=1)
categories = kws_matches.select("cat_web").unique().to_series()
data = data.select("ID", "url").filter(pl.col("url")!= "").group_by("ID").agg(pl.col("url").first()).collect(engine = "streaming")
kws_matches = kws_matches.join(data, on = "ID")


In [12]:
import pandas as pd
cat_ids = []
chosen = []
for cat in categories:
    companies = kws_matches.filter(pl.col("cat_web") == cat).filter(~pl.col("ID").is_in(chosen)).unique("ID")\
    .head(10)
    urls = companies["url"].to_list()
    ids = companies["ID"].to_list()
    
    chosen.append("ids")

    result = {"category": cat, "url": urls, "ID": ids}
    result = pd.DataFrame(data = result)
    cat_ids.append(result)

result = pd.concat(cat_ids)

In [14]:
pl.from_pandas(result).unique("ID")

category,url,ID
str,str,str
"""Adversarial AI""","""https://www.wicona.com/en/int/""","""CHCHE107157040"""
"""Post-quantum cryptography""","""https://www.lugano.ch/""","""CHCHE189656817"""
"""Adversarial AI""","""https://www.chemgineering.com/""","""CHCHE114897538"""
"""Radar""","""https://adams-armaturen.de/wp/""","""CHCHE107949733"""
"""Atomic clocks""","""https://www.projecthope.org/""","""CHCHE301342031"""
…,…,…
"""High-specification machining p…","""https://isb-foundation.com""","""CHCHE443165982"""
"""Radar""","""https://www.ibsafoundation.org…","""CHCHE494600847"""
"""Novel metamaterials""","""https://sabl.ch""","""CHCHE460134209"""


In [15]:
kws_matches = pl.read_parquet("../data/processed/kw_web_counts.parquet").explode("cat_web").filter(~pl.col("ID").is_in(chosen))
kws_matches = kws_matches.join(data, on = "ID")
null_ids = kws_matches.filter(pl.col("cat_web").is_null()).head(40)
urls = null_ids["url"].to_list()
ids = null_ids["ID"].to_list()
result = {"category": 'None', "url": urls ,"ID": ids}
result = pd.DataFrame(result)

cat_ids.append(result)

In [16]:
result = pd.concat(cat_ids)

In [17]:
result.to_csv("../data/companies_for_review.csv")

In [7]:
import json

with open("../data/processed/companies_for_review.json", "w") as f:
    json.dump(cat_ids, f, indent = 2)

NameError: name 'cat_ids' is not defined

In [10]:
with open("../data/processed/companies_for_review.json") as f:
    data = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
def get_ids(data):
    ids = []
    for d in data:
        for id in d["ID"]:
            ids.append(id)
    return ids

get_ids(data)

NameError: name 'data' is not defined

In [20]:
import polars as pl
data = pl.read_csv("../data/companies_for_review.csv")
data

,category,url,ID
i64,str,str,str
0,"""Satellite positioning and navi…","""https://fem-vd.ch""","""CHCHE132797231"""
1,"""Satellite positioning and navi…","""https://stadtfilter.ch""","""CHCHE114534981"""
2,"""Satellite positioning and navi…","""https://www.swisslm.ch/""","""CHCHE357401616"""
3,"""Satellite positioning and navi…","""https://inergy24.com""","""CHCHE447321927"""
4,"""Satellite positioning and navi…","""https://www.iveco.com/global/r…","""CHCHE496890366"""
…,…,…,…
35,"""None""","""https://www.pandabuns.ch/""","""CHCHE192267600"""
36,"""None""","""https://www.ambuehl-vogelsang.…","""CHCHE426718041"""
37,"""None""","""https://bergsport-gstaad.ch/de…","""CHCHE111842334"""


In [21]:
run = pl.scan_parquet("../data/cleaned/run_2025-11-21/*.parquet")

In [22]:
selected_ids = run.filter(pl.col("ID").is_in(data["ID"])).collect(engine="streaming")

/tmp/ipykernel_174725/1970903931.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  selected_ids = run.filter(pl.col("ID").is_in(data["ID"])).collect(engine="streaming")


In [23]:
selected_ids.group_by("ID").agg(pl.col("text").str.join("\n")).lazy().sink_parquet("/home/msalvetti/llm_classification/data/subset/run1.parquet", row_group_size=200)

In [8]:
selection = pl.scan_parquet("/home/msalvetti/llm_classification/data/subset/run.parquet")#.to_pandas().to_parquet("../data/run.parquet")
data = pl.scan_parquet("../data/cleaned/run_2025-11-21/*.parquet")

In [13]:
urls = data.join(selection, on = "ID").select('ID', "url").group_by("ID").agg(pl.col("url").first()).collect(engine = "streaming")
urls.write_csv("../data/companies_for_review.csv")

In [ ]:
urls.join(categories)

ID,url
str,str
"""CHCHE103976091""","""https://www.infotech-automatio…"
"""CHCHE108144560""",""""""
"""CHCHE113331004""",""""""
"""CHCHE109183143""","""https://www.m-ici.ch"""
"""CHCHE435060401""","""https://gaffuri1914.ch/it-ch/"""
…,…
"""CHCHE368449892""","""https://paarberatung.zuerich/"""
"""CHCHE109725239""","""https://keller-pressure.com/en…"
"""CHCHE109526935""","""https://presserat.ch/category/…"


In [18]:
kws_matches = pl.read_parquet("../data/processed/kw_web_counts.parquet").explode("cat_web").filter(pl.col("ID").is_in(urls["ID"]))

/tmp/ipykernel_41445/3484479656.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  kws_matches = pl.read_parquet("../data/processed/kw_web_counts.parquet").explode("cat_web").filter(pl.col("ID").is_in(urls["ID"]))


In [27]:
kws_matches.join(urls, on = "ID").select("ID", "url", "cat_web").group_by("ID").agg(
    pl.col("url").first(),
    pl.col("cat_web")
    )#.write_csv("../data/companies_for_review.csv")

ID,url,cat_web
str,str,list[str]
"""CHCHE421804779""","""https://www.hotelcontinental.c…","[""Biofuels"", ""Advanced magnets and superconductors"", … ""Nuclear medicine and radiation therapy""]"
"""CHCHE116271025""","""https://www.emag.com/""","[""Machine learning (including neural networks and deep learning)"", ""High-specification machining processes"", … ""Electric batteries""]"
"""CHCHE106777033""","""https://ch.fishersci.com/ch/de…","[""Genetic engineering"", ""Advanced protection"", … ""AI algorithms and hardware accelerators""]"
"""CHCHE101519512""","""https://symalit.ch/""","[""Wide and ultrawide bandgap semiconductors"", ""Protective cyber security technologies""]"
"""CHCHE189795034""","""https://www.besi.com/""","[""Advanced protection"", ""Autonomous underwater vehicles"", … ""photovoltaics""]"
…,…,…
"""CHCHE105996472""","""https://teboag.ch""",[null]
"""CHCHE105409785""","""https://www.florin-ag.ch/""","[""Biological manufacturing"", ""Critical minerals extraction and processing"", … ""Biofuels""]"
"""CHCHE108694582""","""https://www.wirdgroup.com/""","[""Machine learning (including neural networks and deep learning)"", ""Distributed ledgers"", … ""Advanced data analytics""]"


In [31]:
kws_matches.select("cat_web").unique().to_series().to_list()

['Autonomous systems operation technologies',
 'Critical minerals extraction and processing',
 'Novel antibiotics and antivirals',
 'photovoltaics',
 'Multispectral and hyperspectral imaging sensors',
 'Space launch systems (including launch vehicles and supporting infrastructure)',
 'AI algorithms and hardware accelerators',
 'nuclear waste management and recycling',
 'Coatings',
 'Advanced protection',
 'Advanced explosives and energetic materials',
 'Natural language processing (including speech and text recognition and analysis)',
 'Quantum sensors',
 'Waste water - Drinking water',
 'Photonic sensors',
 'Nuclear medicine and radiation therapy',
 'Distributed ledgers',
 'supercapacitors',
 'Hypersonic detection and tracking',
 'Genetic engineering',
 'Sonar and acoustic sensors',
 'Biological manufacturing',
 None,
 'Mesh and infrastructure independent networks',
 'Genomic sequencing and analysis',
 'Air independent propulsion',
 'Autonomous underwater vehicles',
 'Continuous flow 